## Running Group Finder

This notebook will call the functions to preprocess, run group finder, and run postprocessing code to build up a GroupCatalog object, which mostly wraps a pandas DataFrame containing the resulting group catalog data.

After running this on a given GroupCatalog definition, a serialized (via pickle) version of the GroupCatalog object will exist which can be deserialized elsewhere for analysis. See post_plots.ipynb for that.

In [ ]:
import sys
import matplotlib.pyplot as plt
from astropy.table import Table,join
import astropy.io.fits as fits

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from groupcatalog import *
import catalog_definitions as cat
from pyutils import *
from dataloc import *
import plotting as pp
%load_ext autoreload
%autoreload 2

In [ ]:
datasets_to_run: list[GroupCatalog] = []
#datasets_to_run.extend(cat.sdss_list)
#datasets_to_run.extend(cat.uchuu_list)
#datasets_to_run.extend(cat.mxxl_list)
datasets_to_run.extend(cat.bgs_sv3_list)  
#datasets_to_run.extend(cat.bgs_y1_list)  
#datasets_to_run.extend(cat.bgs_y3_list)  
#datasets_to_run.extend(cat.bgs_aux_list)

#gc = cat.bgs_y1_hybrid8_mcmc

datasets_to_run.extend([
    #cat.bgs_y1_pzp_2_4_c2, # groupfind() took 46.82 sec. (47.11)
    #cat.bgs_y1_pzp_2_4_c2_serial, # groupfind() took 77.91 sec (74.34)
    #cat.bgs_y1_pzp_2_4_c2_withrc, # groupfind() took 54.46 sec
    #cat.bgs_y1_hybrid8_mcmc

])

# 7/16 fix to use fsf abs mags made C2 go from 199 to 239 chi squared

for d in datasets_to_run:
    d = deserialize(d)
    #d.preprocess()
    success = d.run_group_finder(popmock=True, profile=False, silent=False, serial=False)
    if not success:
        print(f"Group finder failed for {d.name}")
        continue
    d.calc_wp_for_mock()
    d.postprocess()
    d.dump()
    d.chisqr()

In [ ]:
#gc = deserialize(cat.bgs_y1_hybrid8_mcmc)

for d in datasets_to_run:
    gc = deserialize(d)
    pp.proj_clustering_plot(gc)
    pp.lsat_data_compare_plot(gc)
    pp.hod_plot(gc)
    pp.single_plots(gc)

In [ ]:
#bgs_sv3_pz_2_4_10p = deserialize(cat.bgs_sv3_pz_2_4_10p)
#bgs_sv3_pz_2_4_10p.add_jackknife_err_to_proj_clustering(with_extra_randoms=True, for_mag_bins=False)
#serialize(bgs_sv3_pz_2_4_10p)